In [ ]:
import pandas as pd

In [ ]:
data = pd.read_parquet("aggregate_data.parquet")
data["count_views"] = data["<5"] + data["<30"] + data["<60"] + data[">60"]
data["published_yearmonth"] = data["published_at_local"].map(lambda x: f"{x.year}-{str(x.month).zfill(2)}")

data["published_yearmonth"] = pd.to_datetime(data["published_yearmonth"])

data

In [ ]:
data.sort_values(by="count_views", ascending=False).head(8)

In [ ]:
data["count_views"].quantile(.98)

In [ ]:
data_98_percentile = data[data["count_views"]<data["count_views"].quantile(.98)]

In [ ]:
data_98_percentile_yearmonth = data_98_percentile.groupby(["published_yearmonth", "publisher_id", "locality", "topic", "genre"]).sum().reset_index()
data_98_percentile_yearmonth

In [ ]:
data_98_percentile_yearmonth.to_parquet("aggregate_data_98_percentile_yearmonth.parquet")

In [ ]:
data_98_percentile_yearmonth.groupby(["published_yearmonth", "topic"]).sum().reset_index()

In [ ]:
data_98_percentile_yearmonth.groupby(["published_yearmonth", "topic"]).sum()

In [ ]:
totalviews_yearmonth = data_98_percentile_yearmonth.groupby(["published_yearmonth"]).sum().reset_index()[["published_yearmonth","count_views"]]
totalviews_yearmonth = totalviews_yearmonth.rename(columns={"count_views": "count_totals"})
totalviews_yearmonth.head()

In [ ]:
data_98_percentile_yearmonth_totals_absolute = data_98_percentile_yearmonth.groupby(["published_yearmonth", "topic"]).sum().reset_index().merge(totalviews_yearmonth, on="published_yearmonth")
data_98_percentile_yearmonth_totals_absolute["count_relative"] = data_98_percentile_yearmonth_totals_absolute["count_views"]/data_98_percentile_yearmonth_totals_absolute["count_totals"]*100
data_98_percentile_yearmonth_totals_absolute

In [ ]:
data_98_percentile_yearmonth.groupby(["published_yearmonth"]).sum().reset_index()[["published_yearmonth","count_views"]]

In [ ]:
data_98_percentile_year_agg = data_98_percentile_yearmonth.copy()
data_98_percentile_year_agg["published_year"] = data_98_percentile_year_agg.published_yearmonth.map(lambda x: x.year)
data_98_percentile_year_agg

In [ ]:
data_98_percentile_year_agg.groupby(["published_year"]).sum().reset_index()

In [ ]:
totalviews_year = data_98_percentile_year_agg.groupby(["published_year"]).sum()
columns = totalviews_year.columns
columns_new_dict = {column: f"{column}_sum" for column in columns}
totalviews_year = totalviews_year.reset_index()
totalviews_year = totalviews_year.rename(columns=columns_new_dict)
totalviews_year.head()

In [ ]:
data_98_percentile_year_agg

In [ ]:
data_98_percentile_year_agg.groupby(["published_year", "published_yearmonth", "topic"]).sum().reset_index().merge(totalviews_year, on="published_year")

In [ ]:
data_98_percentile_year_agg = data_98_percentile_year_agg.groupby(["published_year", "published_yearmonth", "topic"]).sum().reset_index().merge(totalviews_year, on="published_year")
#data_98_percentile_year_agg["count_relative"] = data_98_percentile_year_agg["count_views"]/data_98_percentile_year_agg["count_totals"]*100
data_98_percentile_year_agg

In [ ]:
data_98_percentile_year_agg["<5"] / data_98_percentile_year_agg["<5_sum"] * 100

for column in columns:
    data_98_percentile_year_agg[f"{column}_rel"] = data_98_percentile_year_agg[column] / data_98_percentile_year_agg[f"{column}_sum"] * 100

In [ ]:
data_98_percentile_year_agg[["topic","<5_rel", "<30_rel", "<60_rel", ">60_rel"]].melt(id_vars="topic")

In [ ]:
data_98_percentile_yearmonth.groupby(["published_yearmonth"]).sum().reset_index()[["published_yearmonth","count_views"]]

In [ ]:
pd.unique(data_98_percentile_yearmonth.topic)

In [ ]:
sorted(pd.unique(data_98_percentile_yearmonth.topic))

In [ ]:
filterlist = ['Justiz/Kriminalität', 'Politik', 'Sport: Nicht-Fußball',]

In [ ]:
data_98_percentile_yearmonth[data_98_percentile_yearmonth["topic"].isin(filterlist)]

In [ ]:
[{"label": listitem, "value": listitem} for listitem in pd.unique(data_98_percentile_yearmonth.topic)]

In [ ]:
pd.unique(data_98_percentile_yearmonth.topic)[1]

In [ ]:
    groupby_filter_df = filter_df.groupby(["published_yearmonth", "topic"]).sum().reset_index()
